In [2]:
from sklearn.ensemble import RandomForestClassifier
from collections import deque
import numpy as np


In [16]:
def generate_fraud_data_block(num_samples, num_features):
    """
    Tạo một khối dữ liệu với num_samples mẫu và num_features đặc trưng.
    Nhãn về phát hiện gian lận được sinh ngẫu nhiên với xác suất nhất định.
    """
    # Tạo dữ liệu đặc trưng ngẫu nhiên
    features = np.random.rand(num_samples, num_features)
    
    # Sinh nhãn (0 = không gian lận, 1 = gian lận), với xác suất xảy ra gian lận thấp hơn
    labels = np.random.choice([0, 1], size=num_samples, p=[0.9, 0.1])
    
    # Kết hợp các đặc trưng và nhãn
    block = list(zip(features, labels))
    return block

def generate_fraud_data_stream(num_blocks, num_samples_per_block, num_features):
    """
    Tạo luồng dữ liệu phát hiện gian lận gồm nhiều khối.
    """
    data_stream = []
    for i in range(num_blocks):
        block = generate_fraud_data_block(num_samples_per_block, num_features)
        data_stream.append(block)
    return data_stream

# Ví dụ tạo 5 block dữ liệu phát hiện gian lận
num_blocks = 5
num_samples_per_block = 10
num_features = 10

fraud_data_stream = generate_fraud_data_stream(num_blocks, num_samples_per_block, num_features)

# Hiển thị block đầu tiên để kiểm tra
S = fraud_data_stream

In [15]:
def train_random_forest_classifier(X, y, n_estimators=10):
    """
    Huấn luyện một bộ phân loại Random Forest trên dữ liệu X, y.
    n_estimators: Số lượng cây trong rừng ngẫu nhiên.
    """
    rf = RandomForestClassifier(n_estimators=n_estimators)
    rf.fit(X, y)
    return rf


def linear_regression(x, y):
    """Tính hồi quy tuyến tính"""
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    return m, c


def adaptive_ensemble_size(C, sample, Y,  epsilon=0.01, min_classifiers=3):
    return

def tendency_prediction(probability_list, Y, epsilon=0.01):
    predicted_probabilities = []
    for i in range(len(Y)):
        li = probability_list[i]
        
        x = np.arange(1, len(li) + 1)
        y = np.array(li)
        
        # Tính hồi quy tuyến tính f(li, g(z))
        slope, intercept = linear_regression(x, y)
        
        # Dự đoán giá trị tiếp theo g(|li| + 1)
        next_value = slope * (len(li) + 1) + intercept
        li.append(next_value)
        
        # Tính xác suất có trọng số bằng phương pháp trung bình có trọng số
        weighted_prob = sum([li[x] * (1 + x * epsilon) for x in range(len(li))]) / len(li)
        predicted_probabilities.append(weighted_prob)
    
    Ps = Y[np.argmax(predicted_probabilities)]
    
    return Ps

def process_data_stream(S, m, k, epsilon=0.01):
    """
    Xử lý luồng dữ liệu S theo thuật toán với bộ phân loại Random Forest yếu.
    - S: Luồng dữ liệu (một danh sách các khối Bi)
    - m: Kích thước tối đa của tập hợp bộ phân loại C
    - k: Kích thước tối thiểu của C để tiến hành dự đoán
    """
    C = deque(maxlen=m) 
    Pre = []
    for Bi in S:
        bl1 = []
        X = np.array([sample[0] for sample in Bi])  
        y = np.array([sample[1] for sample in Bi])  
        Ci = train_random_forest_classifier(X, y)
        while len(C) < k:
            C.append(Ci)
        if len(C) < m:
            C.append(Ci)
        else:
            C.append(Ci) 
        for sample, true_label in Bi:
            selected_classifiers = adaptive_ensemble_size(C, sample, true_label)
            predicted_class = tendency_prediction(selected_classifiers, true_label, epsilon)
            bl1.append(predicted_class)
        Pre.append(bl1)
    return C, Pre

m = 15
k = 3

process_data_stream(S, m, k)

TypeError: object of type 'numpy.int32' has no len()

In [5]:
def linear_regression(x, y):
    """Tính hồi quy tuyến tính"""
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    return m, c

def tendency_prediction(probability_list, Y, epsilon=0.01):
    """
    Thực hiện cơ chế dự đoán xu hướng
    Input:
    - probability_list: Danh sách xác suất cho mỗi mẫu
    - Y: Danh sách các lớp trong luồng dữ liệu
    - epsilon: Hệ số trọng số (mặc định là 0.01)
    Output:
    - Ps: Lớp dự đoán cho mẫu
    """
    predicted_probabilities = []
    for i in range(len(Y)):
        li = probability_list[i]
        
        x = np.arange(1, len(li) + 1)
        y = np.array(li)
        
        # Tính hồi quy tuyến tính f(li, g(z))
        slope, intercept = linear_regression(x, y)
        
        # Dự đoán giá trị tiếp theo g(|li| + 1)
        next_value = slope * (len(li) + 1) + intercept
        li.append(next_value)
        
        # Tính xác suất có trọng số bằng phương pháp trung bình có trọng số
        weighted_prob = sum([li[x] * (1 + x * epsilon) for x in range(len(li))]) / len(li)
        predicted_probabilities.append(weighted_prob)
    
    Ps = Y[np.argmax(predicted_probabilities)]
    
    return Ps

probability_list = [
    [0.6, 0.65, 0.7],  # Xác suất cho lớp 1
    [0.4, 0.35, 0.3],  # Xác suất cho lớp 2
]
Y = ['Class 1', 'Class 2']

# Gọi hàm dự đoán
predicted_class = tendency_prediction(probability_list, Y)
print(f'Dự đoán lớp: {predicted_class}')


Dự đoán lớp: Class 1


In [12]:
a = []
b = []
a.append(b)
a[0].append(1)
print(a)

[[1]]
